In [1]:
import json
import random
import pandas as pd
import pymysql
from faker import Faker
from datetime import datetime, timedelta,time 
from dateutil.relativedelta import relativedelta  # Import relativedelta for date calculations
from dateutil.parser import parse  # Import parse for date string to datetime conversion
import pymysql
from datetime import datetime, timezone
from datetime import datetime
import pytz
import hashlib

pst = pytz.timezone('America/Los_Angeles')

In [3]:
def deposit_funds(user_id, account_type):
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {user_id} AND AccountType = '{account_type}'")
        account_info = cursor.fetchone()

    if account_info:
        amount = float(input("Enter amount: "))

        current_balance = account_info[1]

        confirm = input(f"Are you sure you want to deposit ${amount} into your {account_type} account? (Yes/No): ")
        if confirm.lower() == 'yes':
            with connection.cursor() as action_cursor:
                action_cursor.execute("COMMIT")
                action_cursor.fetchall()
                action_cursor.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;")
                action_cursor.execute("START TRANSACTION;")
                action_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {user_id} AND AccountType = '{account_type}'")
                account_info = action_cursor.fetchone()
                current_balance = account_info[1]
                try:
                    new_balance = current_balance + amount
                    pst_now = datetime.now(pst)
                    action_cursor.execute(f"UPDATE Account SET CurrentBalance = {new_balance} WHERE AccountID = {account_info[0]}")
                    #result = 1/0
                    action_cursor.execute("INSERT INTO Transaction (AccountID, TransactionType, Amount, TransactionDate) VALUES (%s, %s, %s, %s)", (account_info[0], 'deposit', amount,pst_now))
                    action_cursor.execute("COMMIT")
                    connection.commit()
                    print("Deposit successful.")
                except Exception as e:
                    connection.rollback()
                    print("Error occurred during deposit. Rolling back changes.")
                    print("Error:", e)
        else:
            print("Deposit canceled by the user.")
    else:
        print(f"No {account_type} account found for the provided User ID.")

def withdraw_funds(user_id, account_type):
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {user_id} AND AccountType = '{account_type}'")
        account_info = cursor.fetchone()

    if account_info:
        amount = float(input("Enter amount: "))

        current_balance = account_info[1]

        confirm = input(f"Are you sure you want to withdraw ${amount} from your {account_type} account? (Yes/No): ")
        if confirm.lower() == 'yes':
            if amount > current_balance or amount <= 0:
                
                print("Withdrawal amount is invalid. Transaction cancelled.")
            else:
                with connection.cursor() as action_cursor:
                    action_cursor.execute("COMMIT")
                    action_cursor.fetchall()
                    action_cursor.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;")
                    action_cursor.execute("START TRANSACTION")
                    action_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {user_id} AND AccountType = '{account_type}'")
                    account_info = action_cursor.fetchone()
                    current_balance = account_info[1]
                    try:
                        new_balance = current_balance - amount
                        pst_now = datetime.now(pst)
                        action_cursor.execute(f"UPDATE Account SET CurrentBalance = {new_balance} WHERE AccountID = {account_info[0]}")
                        action_cursor.execute("INSERT INTO Transaction (AccountID, TransactionType, Amount, TransactionDate) VALUES (%s, %s, %s, %s)", (account_info[0], 'withdraw', amount,pst_now))
                        action_cursor.execute("COMMIT")
                        connection.commit()
                        print("Withdrawal successful.")
                    except Exception as e:
                        connection.rollback()
                        print("Error occurred during withdrawal. Rolling back changes.")
                        print("Error:", e)
        else:
            print("Withdrawal canceled by the user.")
    else:
        print(f"No {account_type} account found for the provided User ID.")

def transfer_within_accounts(sender_id, sender_account_type, recipient_account_type, amount):
    with connection.cursor() as sender_cursor:
        sender_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {sender_id} AND AccountType = '{sender_account_type}'")
        sender_account = sender_cursor.fetchone()

    with connection.cursor() as recipient_cursor:
        recipient_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {sender_id} AND AccountType = '{recipient_account_type}'")
        recipient_account = recipient_cursor.fetchone()

    if sender_account and recipient_account:
        sender_balance = sender_account[1]

        confirm = input(f"Are you sure you want to transfer ${amount} from your {sender_account_type} account to your {recipient_account_type} account? (Yes/No): ")
        if confirm.lower() == 'yes':
            if amount > sender_balance or amount <= 0:
                print("Transfer amount is invalid. Transaction cancelled.")
            else:
                with connection.cursor() as action_cursor:
                    action_cursor.execute("COMMIT")
                    action_cursor.fetchall()
                    action_cursor.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;")
                    action_cursor.execute("START TRANSACTION")
                    action_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {sender_id} AND AccountType = '{sender_account_type}'")
                    sender_account = action_cursor.fetchone()
                    action_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {sender_id} AND AccountType = '{recipient_account_type}'")
                    recipient_account = action_cursor.fetchone()
                    try:
                        new_sender_balance = sender_balance - amount
                        action_cursor.execute(f"UPDATE Account SET CurrentBalance = {new_sender_balance} WHERE AccountID = {sender_account[0]}")
                        pst_now = datetime.now(pst)

                        action_cursor.execute("INSERT INTO Transaction (AccountID, TransactionType, Amount, TransactionDate, TransferAccountID) VALUES (%s, %s, %s, %s , %s)", (sender_account[0], 'transfer', amount,pst_now, recipient_account[0]))

                        new_recipient_balance = recipient_account[1] + amount
                        action_cursor.execute(f"UPDATE Account SET CurrentBalance = {new_recipient_balance} WHERE AccountID = {recipient_account[0]}")
                        action_cursor.execute("COMMIT")
                        connection.commit()
                        print("Internal transfer successful.")
                    except Exception as e:
                        connection.rollback()
                        print("Error occurred during internal transfer. Rolling back changes.")
                        print("Error:", e)
        else:
            print("Internal transfer canceled by the user.")
    else:
        print("One or both accounts not found.")

def transfer_to_external(sender_id, recipient_id, recipient_account_type, amount):
    # Verify sender's account information
    with connection.cursor() as sender_cursor:
        sender_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {sender_id}")
        sender_account = sender_cursor.fetchone()

    # Verify recipient's account information
    with connection.cursor() as recipient_cursor:
        recipient_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {recipient_id} AND AccountType = '{recipient_account_type}'")
        recipient_account = recipient_cursor.fetchone()

    if sender_account and recipient_account:
        sender_balance = sender_account[1]

        confirm = input(f"Are you sure you want to transfer ${amount} to User ID {recipient_id}? (Yes/No): ")
        if confirm.lower() == 'yes':
            if amount > sender_balance or amount <= 0 or amount> 5000:
                print("Transfer amount is invalid. Transaction cancelled.")
            else:
                with connection.cursor() as action_cursor:
                    action_cursor.execute("COMMIT")
                    action_cursor.fetchall()
                    action_cursor.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;")
                    action_cursor.execute("START TRANSACTION")
                    action_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {sender_id}")
                    sender_account = action_cursor.fetchone()
                    action_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {recipient_id} AND AccountType = '{recipient_account_type}'")
                    recipient_account = action_cursor.fetchone()
                    try:
                        new_sender_balance = sender_balance - amount
                        action_cursor.execute(f"UPDATE Account SET CurrentBalance = {new_sender_balance} WHERE AccountID = {sender_account[0]}")
                        pst_now = datetime.now(pst)

                        action_cursor.execute("INSERT INTO Transaction (AccountID, TransactionType, Amount, TransactionDate, TransferAccountID) VALUES (%s, %s, %s, %s ,%s )", (sender_account[0], 'Send', amount, pst_now,recipient_account[0]))
                        action_cursor.execute("INSERT INTO Transaction (AccountID, TransactionType, Amount, TransactionDate, TransferAccountID) VALUES (%s, %s, %s, %s ,%s )", (recipient_account[0], 'Receive', amount, pst_now,sender_account[0]))

                        new_recipient_balance = recipient_account[1] + amount
                        action_cursor.execute(f"UPDATE Account SET CurrentBalance = {new_recipient_balance} WHERE AccountID = {recipient_account[0]}")
                                       
                        action_cursor.execute("COMMIT")
                        connection.commit()
                        print("External transfer successful.")
                    except Exception as e:
                        connection.rollback()
                        print("Error occurred during external transfer. Rolling back changes.")
                        print("Error:", e)
        else:
            print("External transfer canceled by the user.")
    else:
        print("One or both users don't have an account.")
# Banking transactions function
def banking_transactions(action, user_id):
    with connection.cursor() as sender_cursor:
        sender_cursor.execute(f"SET AUTOCOMMIT=0;")
        sender_account = sender_cursor.fetchall()
    
    if action == 'deposit':
        account_type = input("Enter Account Type (e.g., checking, savings): ").lower()
        deposit_funds(user_id, account_type)
    elif action == 'withdraw':
        account_type = input("Enter Account Type (e.g., checking, savings): ").lower()
        withdraw_funds(user_id, account_type)
    elif action == 'transfer':
        within_or_external = input("Enter transfer type (within/external): ").lower()
        with connection.cursor() as sender_cursor:
            sender_cursor.execute(f"SELECT AccountID, CurrentBalance FROM Account WHERE PersonID = {user_id}")
            sender_account = sender_cursor.fetchall()
        if within_or_external == 'within':
            if len(sender_account) == 1:
                print('You only have one account. Could not make a transfer')
            else:
                from_account_type = input("Enter Account Type to transfer from (e.g., checking, savings): ").lower()
                to_account_type = input("Enter Account Type to transfer to (e.g., checking, savings): ").lower()
                amount = float(input("Enter transfer amount: "))
                # Process transfer between user's accounts
                # transfer_within_accounts(user_id, from_account_type, to_account_type, amount)
        elif within_or_external == 'external':
            recipient_account = str(input("Enter Recipient's Account: "))
            with connection.cursor() as recipient_cursor:
                recipient_cursor.execute(f"SELECT AccountID, AccountType FROM Account WHERE AccountNumber = '{recipient_account}'")
                recipient_account = recipient_cursor.fetchone()
            recipient_id =  recipient_account[0]
            recipient_account_type =  recipient_account[1]
            amount = float(input("Enter transfer amount: "))
            # Process transfer to different user's accounts
            transfer_to_external(user_id, recipient_id, recipient_account_type, amount)
        else:
            print("Invalid transfer type. Please enter 'within' or 'external'.")
    else:
        print("Invalid action entered. Please enter deposit, withdraw, or transfer for this transaction.")
# Establish a connection to the database
connection = pymysql.connect(
    host='34.94.165.202',
    user='root',  # enter your username
    password='thereisnopassword',  # enter your password 
    database='banking_system'
)
# Login function
print('***************************************************************')
def show_user_transactions(email, connection):
    
    try:
        with connection.cursor() as cursor:
            
            req = input("How many days of transactions do you want to view?")
            query = f"""
                SELECT T.TransactionDate, A.AccountType, T.TransactionType, CONCAT('$', T.Amount) AS Amount,
                        A1.AccountNumber, P1.Email
                FROM Transaction T
                LEFT JOIN Account A ON T.AccountID = A.AccountID
                LEFT JOIN Person P ON A.PersonID = P.PersonID
                LEFT JOIN Account A1 ON T.TransferAccountID = A1.AccountID
                LEFT JOIN Person P1 ON A1.PersonID = P1.PersonID
                WHERE P.Email = '{email}'
                ORDER BY T.TransactionDate DESC LIMIT {req};
            """
            
            
            cursor.execute(query)
            info = cursor.fetchall()
            if len(info) == 0:
                print("No transactions found.")
            else:
                print('--------------- Transaction ------------------ ')
                df = pd.DataFrame(info, columns = ['Date','Account', 'Type', 'Amount', 'AccountID', 'Email'])
                df.Type = df.Type.str.capitalize()
                pd.set_option('max_colwidth', 1000)
                print(df.to_string(index=False))
            
    except Exception as e:
        print("Transaction error:", e)
def show_user_balance(email, connection):
   
    query = f"""
        SELECT A.AccountNumber, A.AccountType, A.CurrentBalance
        FROM Account A
        JOIN Person P ON A.PersonID = P.PersonID
        WHERE P.Email = '{email}';
    """

    try:
        with connection.cursor() as cursor:
            
            cursor.execute("START TRANSACTION;")
            cursor.execute(query)
            info_account = cursor.fetchall()
            df_acc = pd.DataFrame(info_account, columns = ['Account Numbe','Account Type', 'Current Balance'])
            print( df_acc)
            
    except Exception as e:
        print("Error fetching balance:", e)


def login(connection):
    while True:
        req = input("Welcome, please type 'login' or 'signup', or 'quit' to end: ").lower()

        if req == 'quit':
            print("Exiting the system.")
            break

        while req not in ['login', 'signup']:
            print("Invalid input.")
            req = input("Please enter 'login', 'signup', or 'quit': ").lower()

        if req == 'login':
            email = input("Enter your email: ")
            password = input("Enter your password: ")

            hashed_password = hashlib.sha256(password.encode()).hexdigest()

            with connection.cursor() as cursor:
                cursor.execute("SELECT * FROM UserCredentials WHERE Email = %s", email)
                user = cursor.fetchone()

                if user and user[2] == hashed_password:
                    print("Login successful!")

                    # Show user balance
                    show_user_balance(email, connection)

                    while True:
                        # Show user transactions
                        print('***********************************')
                        print('Options: ')
                        print('1. View Transaction')
                        print('2. Deposit')
                        print('3. Withdraw')
                        print('4. Transfer')
                        print('5. Logout')
                        print('***********************************')
                        action = int(input("Please enter option number : "))

                        if action ==  5 :
                            print("Logout successful!")
                            break
                        elif action == 1:
                            
                            show_user_transactions(email, connection)
                            continue_action = input("Do you want to continue? (yes/no): ").lower()
                            if continue_action == 'no':
                                print("Logout successful!")
                                break
                        elif action ==2:
                            banking_transactions('deposit', user[0])
                            print('-----------------------------------')
                            # Show user current account balance
                            show_user_balance(email, connection)

                            continue_action = input("Do you want to continue? (yes/no): ").lower()
                            if continue_action == 'no':
                                print("Logout successful!")
                                break
                        elif action ==3:
                            banking_transactions('withdraw', user[0])
                            print('-----------------------------------')
                            # Show user current account balance
                            show_user_balance(email, connection)

                            continue_action = input("Do you want to continue? (yes/no): ").lower()
                            if continue_action == 'no':
                                print("Logout successful!")
                                break
                        elif action == 4:
                            banking_transactions('transfer', user[0])
                            print('-----------------------------------')
                            # Show user current account balance
                            show_user_balance(email, connection)

                            continue_action = input("Do you want to continue? (yes/no): ").lower()
                            if continue_action == 'no':
                                print("Logout successful!")
                                break
                        else:
                            print("Invalid action. Please enter 'deposit', 'withdraw', 'transfer', or 'quit'.")
                else:
                    print("Invalid email or password. Please try again.")
        elif req == 'signup':no
            # Implement signup functionality here
            print("Signup functionality not implemented yet.")

# The rest of your code...

# Initial login
login(connection)
#show_user_transactions('cras.eget@protonmail.edu', connection)
#qF2Olvj9iY
#id.nunc@yahoo.org
#43136785
#6440

#cras.eget@protonmail.edu

***************************************************************
Welcome, please type 'login' or 'signup', or 'quit' to end: login
Enter your email: cras.eget@protonmail.edu
Enter your password: m8dnpCi34h
Login successful!
  Account Numbe Account Type  Current Balance
0      43136785     Checking          3382.66
1      45564058      Savings          1717.53
***********************************
Options: 
1. View Transaction
2. Deposit
3. Withdraw
4. Transfer
5. Logout
***********************************
Please enter option number : 4
Enter transfer type (within/external): external
Enter Recipient's Account: 71639742
Enter transfer amount: 2.66
Are you sure you want to transfer $2.66 to User ID 1? (Yes/No): Yes
External transfer successful.
-----------------------------------
  Account Numbe Account Type  Current Balance
0      43136785     Checking          3380.00
1      45564058      Savings          1717.53
Do you want to continue? (yes/no): yes
***********************************
Op